#### Importing Libraries

In [58]:
# #Installing recommended libraries
# !pip install opencv-python
# !pip install easyocr
# !pip install tkinter
# !pip install matplotlib
# !pip install pillow
# !pip install pygame
# !pip install datetime
#!pip install mysql.connector

In [59]:
#---CFT[0]
#Importing libraries
import cv2
import easyocr
from tkinter import *
import tkinter as tk
import time
#---

### Step-[1]: Starting with The Basic GUI

In [60]:
#---CFT[1]
#Basic GUI development
root = tk.Tk()
root.title("ID SCANNER")
root.configure(bg="lightblue")
root.resizable(0,0)

width = 480
height = 480

#loaction of the window*****************
scr_width = root.winfo_screenwidth()
scr_ht = root.winfo_screenheight()
xwid = (scr_width/2)-(width/2)
ywid = (scr_ht/2)-(height/2)
#loaction of the window*****************
root.geometry('%dx%d+%d+%d'%(width,height, xwid, ywid))
# mainloop()
#---

''

In [61]:
#---CFT[2]
#Inserting canvas inside GUI-Window
#***********CANVAS-GUI************************************** 
controlCanvas = Canvas(root, width=480,height=80,bg="lightblue")
controlCanvas.pack(anchor='sw', padx=0,pady=0, expand=False, fill=X)

msg  = "                                  Welcome to ID Scanner                                "
helpp = tk.Label(root, text=msg, bg="#2fb4cc", fg="lightblue" , font=('Ariel',12,'bold'))
helpp.pack(side='top',padx=2,pady=4)

canvas2 = Canvas(root, width=480,height=340,bg="black",scrollregion=(0,0,0,0))
canvas2.pack(padx=0,pady=0, side='bottom', expand=True, fill=BOTH)
#**********************************************
# mainloop()
#---

### Step [2] : Developing Console 

In [62]:
#---CFT[3]
#Crearting a console-window
mylist = Listbox(canvas2, width=720, height=320)

#Adding scrollbar
vbar = Scrollbar(mylist, orient='vertical')
vbar.pack(side=RIGHT, fill=Y)
vbar.config(command=mylist.yview)

#Adding help message in console
mylist.insert(END,"This is an employee id scanning tool, that will")
mylist.insert(END,"help you to verify id cards whether they are")
mylist.insert(END,"real or fake.")
mylist.insert(END,"")
mylist.insert(END,"Help:-")
mylist.insert(END,"ADJUST- can be used to adjust your camera")
mylist.insert(END,"or object before starting your scan.")
mylist.insert(END,"")
mylist.insert(END,"SCAN IDCARD:- This command will help you")
mylist.insert(END,"to scan your ID Card.")
mylist.insert(END,"")
mylist.insert(END,"QUIT- will close the application.")
#Help section closed
mylist.config(fg='gray',bg='black', font=('Arial',12,'bold'), yscrollcommand=vbar.set)
mylist.pack(side=BOTTOM, anchor='n', expand=True, fill=BOTH, padx=8, pady=5)
#Console Closed
# mainloop()
#---

### Step-[3]: Control Panel

In [63]:
#---CFT[11]
#Adding control-buttons in contol-panel
#**********************CONTROL-BUTTONS******************************************
#Camera adjusting Control-Button 
btn1 = tk.Button(controlCanvas, width=15,height=2, text="ADJUST", 
                command=lambda: [launch_camera()], bg="skyblue", fg="blue", font=('Arial',8,'bold'))
btn1.place(relx=0,rely=0, x=20, y=18)

#ID Scanner Control-Button
btn2 = tk.Button(controlCanvas, width=15,height=2, text="SCAN IDCARD", 
                 command=lambda: [scan_data(), restart_pb()], bg="lightgreen", fg="green", font=('Arial',8,'bold'))
btn2.place(relx=0,rely=0, x=170, y=18)

#Function to close opened window 
def destroyWin():
    print("Application has been closed!")
    root.destroy()
#Quit Control-Button    
btn3 = tk.Button(controlCanvas, width=15, height=2, text="QUIT", 
                 command=lambda: [destroyWin()] , bg="#992222", fg="red", font=('Arial',8,'bold'))
btn3.place(relx=0,rely=0, x=330, y=18)
#**************************************************************
# mainloop()
#---

### Step-[4]: Testing Camera

In [64]:
#---CFT[6]
def test_camera():
    cam = cv2.VideoCapture(0 + cv2.CAP_DSHOW)
    WIDTH = cam.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
    HEIGHT = cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 280)

    (x, y, w, h) = (60,50,180,140)

    while(True):
        ret, frame = cam.read()
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)

        font = cv2.FONT_HERSHEY_DUPLEX

        cv2.putText(frame,"Please press 'q' after", (46,210), font, 0.6, (255,255,255), 1, cv2.LINE_AA)
        cv2.putText(frame,"adjusting your object.", (46,232), font, 0.6, (255,255,255), 1, cv2.LINE_AA)
        cv2.imshow("Adjust Camera", frame)

        if cv2.waitKey(1)&0xFF == ord("q"):
            break
    cam.release()
    cv2.destroyAllWindows()
    btn1.config(state='normal')

def launch_camera():
    print("\nCamera has been opened. Please adjust your object...\n")
    time.sleep(0.5)
    x = 5
    txt = ("Camera Testing..: {line}".format(line=x))
    btn2.config(state='normal')
    btn3.config(state='normal')
    btn1.config(state='disabled')
    test_camera()

# mainloop()    
#---

### Step-[5]: Processing Scan Status

#### 5.1: Progress Bar

In [65]:
#---[9]
#****************************Progress Bar*************************
status = Label(mylist, text="S c a n n i n g . . .",bg="black",fg="#00f900", font = ('Arial',12,'bold'))

c1 = tk.Canvas(mylist, width=300, height=25, bg="black", highlightthickness=0, bd=0)
t = Listbox(mylist,width=6,height=1,bg="black", fg="#00f900",highlightthickness=0,font = ('Arial',12,'bold'), bd=0)

x1=5
y1=0
x2=20 #width/2
y2=30 #height/2
value = 0

#Clearing console
def clear_screen():
    global x2, value, c1,status
    
    c1.configure(width=0,height=0)
    status.configure(text="",width=0,height=0)
    c1.pack(side=LEFT, anchor='nw', padx=0,pady=0)
    t.configure(width=0,height=0)
    t.pack(side=RIGHT, anchor='se', padx=0, pady=0)
    status.pack(side=BOTTOM,anchor='s',padx=0,pady=0)

    c1.delete('all')
    t.delete(0,END)
    x2 = 0
#launching progress_bar    
def progress_bar():
    global x1,y1,x2,y2, value, running,status

    limit = 300
    status.configure(text="S c a n n i n g . . .",bg="black",fg="#00f900")
    status.pack(side=TOP,anchor='nw', padx=20,pady=40)
    c1.configure(width=300,height=25, bg="black")
    c1.create_rectangle(x1, y1, x2,y2,fill="#00f900")
    c1.pack(side =LEFT, anchor='nw', padx=0,pady=0)
    t.configure(width=6,height=1)
    t.insert(0,str(round(value))+"%")
    x2 += 10
    value += 3
    #Replaying canvas afer 0.1 second
    if (x2<=(limit)):
        c1.after(100,progress_bar)
    else:
        running = True
        #If prgress-bar reaches to 100% then, clear screen
        clear_screen()
        #After 1 second of delay, launching detetion-window
        time.sleep(1)
        detection(running)

#mainloop()        
#---

### Step-[5.2]: Actions Need to Be taken

In [66]:
#---CFT[10]
#Scanning processed message
#setting control-buttons' status
def scan_data():
    print('\nScanning is in process...\n')
    time.sleep(1)
    btn1.config(state='normal')
    btn3.config(state='normal')
    btn2.config(state='disabled')
    
#Recallable function    
def restart_pb():
    global x2, value,status,t
    t.delete(0,END)
    value = 0
    c1.delete('all')
    mylist.delete(0,END)
    x2 = 0
    status.pack(side=TOP,anchor='nw',padx=20,pady=40)
    t.pack(side=LEFT, anchor='nw', padx=20, pady=0)
    time.sleep(0.3)
    progress_bar()

#mainloop()
#---

### Step-[6]: Detection On Process

In [67]:
#---CFT[7]
#Calling optical recognition model
def dscan():
    reader = easyocr.Reader(['en'], gpu=False)
    return reader

def quitt():
    print("QUIT button clicked...")
    time.sleep(0.5)
    btn1.config(state='normal')
    btn2.config(state='normal')
    btn3.config(state='disabled')

running = True  # Global flag
num = True
x = 0
#---

In [68]:
#---CFT[8]
#Detection script
import mysql.connector
mydatabase = mysql.connector.connect(
    host='localhost',
    user='root',
    password = 'root',
    database = 'employeeinfosystem'
)
    
def detection(running):
    r = 0;
    s = 0;
    reader = dscan()
    #Trying to access camera
    cam = cv2.VideoCapture(0 + cv2.CAP_DSHOW)
    
    #Fetching width-height of camera
    WIDTH = cam.get(cv2.CAP_PROP_FRAME_WIDTH)
    HEIGHT = cam.get(cv2.CAP_PROP_FRAME_HEIGHT)
    
    width = 320
    height = 140
    #loaction of the window*****************
    scr_width = root.winfo_screenwidth()
    scr_ht = root.winfo_screenheight()
    xwid = (scr_width/2)-(width/2)
    ywid = (scr_ht/2)-(height/2)
    #loaction of the window*****************
    
    #Setting custom width-height of of the camera window
    cam.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    
    #setting co-ordinates, and width-height
    (x, y, w, h) = (60,50,180,140)
    
    reader = easyocr.Reader(['en'], gpu=False)
    
    #Data found or not
    def found():
        insertData("RESULT FOUND!")
    def notFound():
        insertData("DATA NOT FOUND uu!")        
    time.sleep(1)
    
    while(running):
        r += 1
        s += 1
        #Reading camera
        ret, frame = cam.read()
        #Cropping WindowSize
        frame = frame[y:y+h, x:x+w]
        #Converting ImageColor
        img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #Sending data to ocr-reader model
        result = reader.readtext(img_gray)
        #Check if data found
        if s<2:
            if (len(result) != 0):
                found()
            else:
                notFound()
        #Restoring image color        
        img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #If detection in multiple lines
        for detection in result:
            top_left = tuple([int(val) for val in detection[0][0]])
            bottom_right = tuple([int (val) for val in detection[0][2]])
            text = detection[1]
            #Sending after detection 
            showData(text)
            
            #check if id found************\
            mydb = mydatabase.cursor(buffered=True)
            cmd = ("SELECT empid FROM employeeinfo")
            mydb.execute(cmd);
            #MySQL database*******************************
            for i in mydb:
                for j in i:
                    if len(j) == 17:
                            insertData(text)    
                    if text == j:
                        cam.release()
                        cv2.destroyAllWindows()
            #************************
            
            #color red:no;green:yes    
            if text != "":
                color = (0,255,0)
            else:
                color = (0,0,255)
            #Drawing rectangle around the detection
            cv2.rectangle(img_gray, top_left, bottom_right, color,2)
        #font, resizing cam-window    
        font = cv2.FONT_HERSHEY_DUPLEX
        extendedFrame = cv2.resize(img_gray,(320,280))
        cv2.putText(extendedFrame,"Please press 'p' to print result.", (6,260), font, 0.6, (255,255,255), 1, cv2.LINE_AA)
        #displaying live detection
        cv2.imshow('CRW', extendedFrame)
        #Command to interrupt connection 
        if cv2.waitKey(1) & 0xFF == ord('p'):
            break
        print("\nText not found!\n")
    #Releasing camera    
    cam.release()
    cv2.destroyAllWindows()
    #Set control button as default state 
    btn2.configure(state='normal')

#mainloop()
#---

### Step-[7.1]: Data Management

In [69]:
#---CFT[5]
#Replacing string to list and back to string
def replaceString(string):
    string2 = ""
    #Convert to List
    ls = []
    i = 0
    for x in string:
        ls.insert(i,str(x))
        i += 1
        
    for j in range(len(ls)):
        #year 1stDigit
        if j == 7:
            if (ls[j]=="l") or (ls[j] == "I"):
                ls.pop(j)
                ls.insert(j,"1")     
                
            if (ls[j]=="Z") or (ls[j] == "z"):
                ls.pop(j)
                ls.insert(j,"2")
                
            if (ls[j]=="S") or (ls[j] =="s"):
                ls.pop(j)
                ls.insert(j,"5")
                
            if (ls[j]=="O") or (ls[j] =="o"):
                ls.pop(j)
                ls.insert(j,"0")
                
            else:
                pass
            
        #year 2ndDigit
        elif j == 8:
            if (ls[j]=="O") or (ls[j] =="o"):
                ls.pop(j)
                ls.insert(j,"0")
                
            if (ls[j]=="l") or (ls[j] =="I"):
                ls.pop(j)
                ls.insert(j,"1")
                
            if (ls[j]=="Z") or (ls[j] =="z"):
                ls.pop(j)
                ls.insert(j,"2")
                
            if (ls[j]=="S") or (ls[j] =="s"):
                ls.pop(j)
                ls.insert(j,"5")
                
            else:
                pass
        
        #emp 1stDigit
        if j ==10:
            if (ls[j]=="l") or (ls[j] == "I"):
                ls.pop(j)
                ls.insert(j,"1")     
                
            if (ls[j]=="Z") or (ls[j] == "z"):
                ls.pop(j)
                ls.insert(j,"2")
                
            if (ls[j]=="S") or (ls[j] =="s"):
                ls.pop(j)
                ls.insert(j,"5")
                
            if (ls[j]=="O") or (ls[j] =="o"):
                ls.pop(j)
                ls.insert(j,"0")
                
            else:
                pass
            
        #emp 2ndDigit
        elif j == 11:
            if (ls[j]=="O") or (ls[j] =="o"):
                ls.pop(j)
                ls.insert(j,"0")
                
            if (ls[j]=="l") or (ls[j] =="I"):
                ls.pop(j)
                ls.insert(j,"1")
                
            if (ls[j]=="Z") or (ls[j] =="z"):
                ls.pop(j)
                ls.insert(j,"2")
                
            if (ls[j]=="S") or (ls[j] =="s"):
                ls.pop(j)
                ls.insert(j,"5")
                
            else:
                pass
        
        #emp unique 1stDigit
        if j ==15:
            if (ls[j]=="z"):
                ls.pop(j)
                ls.insert(j,"Z")  
                
            else:
                pass
            
        #emp 2ndDigit
        elif j == 16:
            if (ls[j]=="z"):
                ls.pop(j)
                ls.insert(j,"Z")
                
            else:
                pass
    
    #convert back to string
    for k in ls:
        string2 = string2+k
    return string2
    
def showData(data):
    mylist.insert(0,"")
    mylist.insert(0, str(data))
    
def insertData(data):
    global mylist
    #Fetching correct data
    newData = replaceString(data)
    mylist.insert(0,"")
    #Displaying data in console
    mylist.insert(0, str(newData))
    #Sending data to displayData function
    displayData(newData)
#---

### Step-[7.2]: Data Validation

In [70]:
#---CFT[4]
#DataValidation
def displayData(idd):
    print("\nData scanned..")
    
    #Calling database start*********************************
    import mysql.connector
    from mysql.connector import Error

    try:
        mydatabase = mysql.connector.connect(
            host='localhost',
            user='root',
            password = 'root',
            database = 'employeeinfosystem'
        )

        if mydatabase.is_connected():
            print("you're connected to the database!")
            mydb = mydatabase.cursor(buffered=True)
            mydb.execute("SELECT * FROM EmployeeInfo")
            #Calling database *********************************
            
            t = [] #Initialize list

            def chk(d):
                print("empid: ",d)
                print("\n")
                
                #MySQL database*******************************
                import mysql.connector
                mydatabase = mysql.connector.connect(
                    host='localhost',
                    user='root',
                    password = 'root',
                    database = 'employeeinfosystem'
                )
                
                mydb = mydatabase.cursor(buffered=True)
            
                eid = str(d)
                cmd = ("SELECT * FROM employeeinfo WHERE empid='{x}';").format(x=eid)
                mydb.execute(cmd);
                #MySQL database*******************************
                
                for i in mydb:
                    for j in i:
                        #print(j)
                        t.insert(len(t)+1, str(j))
                    break
            
                empdb = tuple(t) #Converting list to tuple
                
                #Storing data***************
                empid =  empdb[0]
                fname =  empdb[1]
                gender =  empdb[2]
                dob =  empdb[3]
                depmt =  empdb[4]
                degn =  empdb[5]
                doj =  empdb[6]
                addr =  empdb[7]
                empPhoto =  empdb[8]
                #Storing data*******************
                
                #Destory current window if after data stored
                time.sleep(1)
                root.destroy()
                
                #Getting ready to export data****************
                import EmployeeInfo as empinfo
                from EmployeeInfo import sendData
                
                #Calling EmployeeInformationSystem************
                empinfo.sendData(empid, fname, gender, dob, depmt, degn, doj, addr, empPhoto)

            def insertdata(uinput):
                for i in mydb:
                    for j in i:
                        if j == uinput:  #Checking idd if available
                            chk(j) #if found, sends data to chk
                            
            insertdata(idd)

    except Error as e:
        print("Error while connecting to database: ",e)
#---

In [71]:
#---CFTE[12]
#Packing everything in application
mainloop()



Camera has been opened. Please adjust your object...


Scanning is in process...



Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.



Data scanned..
you're connected to the database!

Text not found!


Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Text not found!


Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Text not found!


Data scanned..
you're connected to the database!

Data scanned..
you're connected to the database!

Data scanned..
you're connected to the databa

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\Python39\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Program Files\Python39\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "C:\Users\Eric.Kibuchi\AppData\Local\Temp\ipykernel_9520\1378671591.py", line 51, in progress_bar
    detection(running)
  File "C:\Users\Eric.Kibuchi\AppData\Local\Temp\ipykernel_9520\1108600533.py", line 82, in detection
    insertData(text)
  File "C:\Users\Eric.Kibuchi\AppData\Local\Temp\ipykernel_9520\1989672139.py", line 132, in insertData
    displayData(newData)
  File "C:\Users\Eric.Kibuchi\AppData\Local\Temp\ipykernel_9520\963516253.py", line 83, in displayData
    insertdata(idd)
  File "C:\Users\Eric.Kibuchi\AppData\Local\Temp\ipykernel_9520\963516253.py", line 81, in insertdata
    chk(j) #if found, sends data to chk
  File "C:\Users\Eric.Kibuchi\AppData\Local\Temp\ipykernel_9520\963516253.py", line

: 